# Importing the Keras libraries and packages

In [20]:
# Importing the Keras libraries and packages

import keras
import sys
import os
import os.path
from keras import backend as K

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers.core import Dropout
from keras.preprocessing.image import ImageDataGenerator

from subprocess import check_output
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from config import Config
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt
import h5py

from PIL import Image


# Import Info and data

Shift + tab # show the documantation

In [21]:
config = Config()

dado = pd.read_csv(config.arquivo_csv).values


# Part 1 - Building the CNN

In [22]:
# Part 1 - Building the CNN

model = Sequential()

model.add(Convolution2D(filters = 32,
                       kernel_size = (5, 5),
                       input_shape = (64, 64, 3),
                       activation = 'relu')) #Primeira camada convolucional com ativação ReLU
model.add(MaxPooling2D(pool_size = (2, 2),
                      strides = 2)) #Pooling Layer de passo 2 que divide a dimensão na metade

model.add(Convolution2D(filters = 32,
                       kernel_size = (5, 5),
                       activation = 'relu'))
#input_shape = (32, 32, 3),
model.add(MaxPooling2D(pool_size = (2,2),
                      strides = 2))

model.add(Convolution2D(filters = 64,
                       kernel_size = (5, 5),
                       activation = 'relu'))
#input_shape = (16, 16, 3),

model.add(Flatten()) #Converte para uma dimensão

# Dense = Full connection
model.add(Dense(activation = 'relu',
               units = 16384)) #FC1
model.add(Dense(activation = 'relu',
               units = 1024)) #FC2
model.add(Dense(activation = 'softmax',
               units = 3))#FC3

# Indexing

### Intervalo das classes:

Graduation [0 - 6991] = 6992 imagens

Picnic [6992 - 13986] = 6994 imagens

Meeting [13986 - 20986] = 7000 imagens

# Compila a arquitetura

In [23]:
#compila a arquitetura
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'],
              loss_weights=None,
              sample_weight_mode=None,
              weighted_metrics=None,
              target_tensors=None)

# Data Augumentation

In [24]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)


# Rescale

In [25]:
test_datagen = ImageDataGenerator(rescale=1./255)

# Utilizando flow_from_directory para treino (60% de cada classe)

In [26]:
train_generator = train_datagen.flow_from_directory(directory=r"/home/medeiros/mini_challenge/data_folder/treino",
                                             target_size = (64, 64),
                                             color_mode = 'rgb',
                                             classes = ['graduation', 'meeting', 'picnic'],
                                             batch_size = 32,
                                             class_mode = 'categorical',
                                             shuffle=True,
                                             seed=42
 ) #Recebe um diretório e caminha por ele
   #com base nos parâmetros fornecidos

Found 12592 images belonging to 3 classes.


# Utilizando flow_from_directory para teste (20% de cada classe)

In [27]:
test_set = test_datagen.flow_from_directory(directory=r"/home/medeiros/mini_challenge/data_folder/teste",
                                           target_size = (64, 64),
                                           color_mode = 'rgb',
                                           classes = ['graduation', 'meeting', 'picnic'],
                                           batch_size = 32,
                                           class_mode = 'categorical')


Found 4197 images belonging to 3 classes.


# Utilizando flow_from_directory para validação (20% de cada classe)

In [28]:
validation_generator = test_datagen.flow_from_directory(directory=r"/home/medeiros/mini_challenge/data_folder/validation",
                                           target_size = (64, 64),
                                           color_mode = 'rgb',
                                           classes = ['graduation', 'meeting', 'picnic'],
                                           batch_size = 32,
                                           class_mode = 'categorical')


Found 4197 images belonging to 3 classes.


# Keras Callbacks

In [29]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')


# model.fit_generator

In [ ]:
history = model.fit_generator(
        train_generator,
        epochs = 50,
        steps_per_epoch = 12592,
        verbose = 1,
        callbacks = [early_stopping],
        validation_data = validation_generator,
        validation_steps = 4179)

# Pós treino / apresentação dos resultados

In [17]:
RefTitleResults = 'L2D3D3G0B1E200_3003001'

In [18]:
###### Plot training & validation accuracy values
plt.figure()
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy ')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()
plt.savefig("AccxEpoch" + RefTitleResults + ".png")

NameError: name 'history' is not defined

<Figure size 432x288 with 0 Axes>

In [32]:
from PIL import Image
#img = Image.open('AccxEpochL2D3D3G0B1E200_3003001.png')
#img

In [ ]:
###### Plot training & validation loss values
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss ')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()
plt.savefig("LossxEpoch" + RefTitleResults + ".png")


In [34]:
#img2 = Image.open('LossxEpochL2D3D3G0B1E200_3003001.png')
#img2

In [35]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
# print(model.evaluate(x_test, y_test))
#print(x_test.shape)
probs = model.predict(validation_generator)
######## keep probabilities for the positive outcome only
probsp = probs  # [:, 1]
# print(probsp.shape)
# print(y_val)
# print(probs)
######## calculate AUC
auc = roc_auc_score(validation_generator, probsp)
print('AUC: %.3f' % auc)

In [ ]:
######## calculate roc curve
fpr, tpr, thresholds = roc_curve(validation_generator, probsp)
